In [27]:
import cv2
import numpy as np
import sklearn
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, MaxPooling2D, Flatten,Lambda,Conv2D,BatchNormalization
from keras import losses
from keras import optimizers
from keras.utils import to_categorical
import os
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

In [173]:

X=[]
Y=[]
def dataset_add(dataset_dir,ylabel,X=X,Y=Y):
    diclist = os.listdir(dataset_dir)
    for filename in diclist:
        img_name = dataset_dir + filename
        img = cv2.imread(img_name)
        img = cv2.resize(img,(80,60))
        X.append(img)
        Y.append(ylabel)
        flipimage = np.fliplr(img)
        X.append(flipimage)
        Y.append(ylabel)

In [174]:
# import the dataset and ad flip images to augment the dataset.
# the dataset mainly comes from two source: ROSbag file from Carla and part of LaRA Traffic Lights Recognition Dataset
dataset_add(dataset_dir='./dataset2/clean/green_2/',ylabel =2)
dataset_add(dataset_dir='./dataset2/clean/red_0/',ylabel =0)
dataset_add(dataset_dir='./dataset2/clean/yellow_1/',ylabel =1)
dataset_add(dataset_dir='./dataset2/clean/unknown_4/',ylabel =3)
dataset_add(dataset_dir='./newsite/red/',ylabel =0)
dataset_add(dataset_dir='./newsite/yellow/',ylabel =1)
dataset_add(dataset_dir='./newsite/green/',ylabel =2)
dataset_add(dataset_dir='./newsite/nol/',ylabel =3)
dataset_add(dataset_dir='./addset/red/',ylabel =0)
dataset_add(dataset_dir='./addset/green/',ylabel =2)
dataset_add(dataset_dir='./addset/abg/',ylabel =1)

In [175]:
y= to_categorical(np.array(Y))
X = np.array(X)

In [176]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.10,random_state=1)  

In [177]:
X.shape

(6884, 60, 80, 3)

In [178]:
y.shape

(6884, 4)

In [180]:
mcheckpoint = ModelCheckpoint('./tl_7.h5', monitor='val_loss', verbose=0, save_best_only=True, 
                              save_weights_only=False, mode='auto', period=1)
reducelr =ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, 
                            verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

In [181]:
model = Sequential()

model.add(BatchNormalization(input_shape=(60,80,3)))
model.add(Conv2D(24,(5,5),strides=(2,2),activation='relu'))

model.add(Conv2D(36,(3,3),strides=(2,2),activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Conv2D(48,(3,3),strides=(2,2),activation='relu'))
model.add(Conv2D(64,(2,2),strides=(1,1),activation='relu'))
model.add(Conv2D(64,(2,2),strides=(1,1),activation='relu'))
model.add(BatchNormalization(axis=1))

model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Dropout(0.4))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Dropout(0.3))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Dropout(0.3))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(4))
model.add(Activation('softmax'))

In [182]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [183]:
model.fit(x=X_train,y=y_train,batch_size=32,epochs=50,verbose=2,
          validation_data=(X_test,y_test),shuffle=True,callbacks =[reducelr,mcheckpoint])

Train on 6195 samples, validate on 689 samples
Epoch 1/50
8s - loss: 1.2663 - acc: 0.4342 - val_loss: 0.9323 - val_acc: 0.5210
Epoch 2/50
4s - loss: 0.9548 - acc: 0.5245 - val_loss: 0.8523 - val_acc: 0.5776
Epoch 3/50
4s - loss: 0.8492 - acc: 0.5727 - val_loss: 0.7734 - val_acc: 0.6226
Epoch 4/50
4s - loss: 0.7542 - acc: 0.6286 - val_loss: 0.6366 - val_acc: 0.7083
Epoch 5/50
4s - loss: 0.6838 - acc: 0.6743 - val_loss: 0.9281 - val_acc: 0.6023
Epoch 6/50
4s - loss: 0.6340 - acc: 0.7073 - val_loss: 0.5433 - val_acc: 0.7736
Epoch 7/50
4s - loss: 0.5202 - acc: 0.7985 - val_loss: 0.2775 - val_acc: 0.8882
Epoch 8/50
4s - loss: 0.3412 - acc: 0.8759 - val_loss: 0.2792 - val_acc: 0.8940
Epoch 9/50
4s - loss: 0.2960 - acc: 0.8973 - val_loss: 0.2414 - val_acc: 0.9028
Epoch 10/50
4s - loss: 0.2821 - acc: 0.9077 - val_loss: 0.1479 - val_acc: 0.9303
Epoch 11/50
4s - loss: 0.2457 - acc: 0.9209 - val_loss: 0.3652 - val_acc: 0.8766
Epoch 12/50
4s - loss: 0.2405 - acc: 0.9285 - val_loss: 1.0744 - val_ac

In [184]:
from keras.models import load_model

In [185]:
tl_model = load_model('./tl_7.h5')


In [186]:
testdir = './testset/'
diclist = os.listdir(testdir)
test_x =[]
test_y= []
imglist=[]
for filename in diclist:
    img_name = testdir+ filename
    img = cv2.imread(img_name)
    img = cv2.resize(img,(80,60))
    #img = img/255.0 -0.5
    imglist.append(img_name)
    test_x.append(img)
    test_y.append(int(filename[-5]))
test_x = np.array(test_x)

In [187]:
result = tl_model.predict_classes(test_x,verbose=0)

In [188]:
result1 =  tl_model.predict(test_x)

In [189]:
print(result)

[1 1 0 0 0 2 2 1 1 0 0 0 0 0]


In [190]:
print(np.array(test_y))

[1 1 0 0 0 2 2 1 1 0 0 0 0 0]


In [191]:
prb = tl_model.predict_proba(test_x)

14/14 [==============================] - 0s


In [192]:
prb[:,2]

array([  2.56142812e-03,   2.26973090e-04,   3.54342829e-15,
         3.03590481e-27,   1.59588159e-11,   1.00000000e+00,
         1.00000000e+00,   5.22797450e-08,   1.82985495e-05,
         2.08936114e-15,   3.25357319e-12,   7.07392882e-17,
         4.42803513e-17,   1.68990065e-11], dtype=float32)

In [49]:
imglist

['./testset/left0197_1.jpg',
 './testset/left0203_1.jpg',
 './testset/left0230_0.jpg',
 './testset/left0264_0.jpg',
 './testset/left0324_0.jpg',
 './testset/left0362_2.jpg',
 './testset/left0452_2.jpg',
 './testset/left0528_1.jpg',
 './testset/left0529_1.jpg',
 './testset/left0539_0.jpg',
 './testset/left0547_0.jpg',
 './testset/left0567_0.jpg',
 './testset/left0577_0.jpg',
 './testset/left0597_0.jpg']